<a href="https://colab.research.google.com/github/Spin-Chemistry-Labs/radicalpy/blob/187-google-colab-tutorials/examples/tutorials/05_incoherent_processes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Tutorial 5 - Incoherent processes


&copy; Lewis M. Antill, 2025

This tutorial aims to introduce kinetic and relaxation superoperators and how they influence radical pair spin dynamics.

In [ ]:
!pip install radicalpy

In [2]:
import ipywidgets as wdg
import numpy as np
import matplotlib.pyplot as plt

import radicalpy as rp
from radicalpy.experiments import magnetic_field_loop
from radicalpy.kinetics import Haberkorn, HaberkornFree
from radicalpy.relaxation import DipolarModulation, RandomFields, SingletTripletDephasing
from radicalpy.simulation import Molecule, LiouvilleSimulation, State

---

### A more complete picture

To fully capture chemical reactions and the influence of the environment on radical pairs, we need to include kinetic and relaxation terms in our spin Hamiltonian.

---

### Kinetic Superoperators

In Hilbert space we used the exponential model simulate a chemical reaction.
However, this model depletes both singlet and triplet populations equally.
We do not have complete control over the reaction pathways.

Now we are in Liouville space, we can use kinetic superoperators to control which state is depleted.
We will use the Haberkorn superoperators:

* Singlet recombination
$$
\hat{\hat{K}} = -\frac{k_{S}}{2} [\hat{Q}_S ⊗ \hat{I} + \hat{I} ⊗ \hat{Q}_S]
$$

* Triplet recombination
$$
\hat{\hat{K}} = -\frac{k_{T}}{2} [\hat{Q}_T ⊗ \hat{I} + \hat{I} ⊗ \hat{Q}_T]
$$

* Free radical / downstream product
$$
\hat{\hat{K}} = -k_{f} [\hat{E} ⊗ \hat{E}]
$$

In [ ]:
def build_sim(hfc1, hfc2):
    radical_1 = Molecule.fromisotopes(name="radical 1", isotopes=["1H"], hfcs=[hfc1])
    radical_2 = Molecule.fromisotopes(name="radical 2", isotopes=["1H"], hfcs=[hfc2])
    return LiouvilleSimulation([radical_1, radical_2])

states = {
    "S": State.SINGLET,
    "T": State.TRIPLET,
    "T+": State.TRIPLET_PLUS,
    "T0": State.TRIPLET_ZERO,
    "T-": State.TRIPLET_MINUS
}

def kinetic_model(model, ks):
    if model == "Haberkorn(S)":
        return [Haberkorn(ks, State.SINGLET)]
    elif model == "Haberkorn(T)":
        return [Haberkorn(ks, State.TRIPLET)]
    elif model == "HaberkornFree":
        return [HaberkornFree(ks)]
    else:
        return []

def nearest_index(arr, value):
    arr = np.asarray(arr)
    return int(np.argmin(np.abs(arr - value)))

def haberkorn_kinetics(sim, init, obs, tmax, Bfield, k, J, D, kinetics_model):
    dB = 0.5
    B0 = np.arange(0.0, 20.0 + 1e-9, dB)
    b_idx = nearest_index(B0, Bfield)

    time = np.arange(0, tmax, 10e-9)

    kinetics = kinetic_model(kinetics_model, k)

    init_state = states[init]
    obs_state  = states[obs]

    H = sim.total_hamiltonian(B0=0, D=D, J=J)
    sim.apply_liouville_hamiltonian_modifiers(H, kinetics)
    rhos = magnetic_field_loop(sim, init_state, time, H, B0, B_axis="z")
    product_probabilities = sim.product_probability(obs_state, rhos)

    sim.apply_hilbert_kinetics(time, product_probabilities, kinetics)
    k = kinetics[0].rate_constant if kinetics else 1.0
    product_yields, product_yield_sums = sim.product_yield(product_probabilities, time, k)

    x = time * 1e6 

    fig = plt.figure()
    plt.plot(x, product_probabilities[b_idx], linewidth=2, label=r"$P_i(t)$")
    plt.fill_between(x, product_yields[b_idx], alpha=0.2, label=r"$\Phi_i$")
    plt.xlabel("Time / $\mu s$", size=14)
    plt.ylabel("Probability", size=14)
    plt.ylim([0, 1])
    plt.legend(fontsize=14)
    fig.set_size_inches([5, 2.5])

    print(f"[{kinetics_model}] PY(B={B0[b_idx]:.1f} mT) = {product_yield_sums[b_idx]:.6g}")

@wdg.interact(
    kinetic_model = ["Haberkorn(S)", "Haberkorn(T)", "HaberkornFree"],
    init=["S", "T", "T+", "T0", "T-"],
    obs=["S", "T", "T+", "T0", "T-"],
    hfc1 = wdg.FloatSlider(min=0.0, max=2.0, step=0.5, value=0),
    hfc2 = wdg.FloatSlider(min=0.0, max=2.0, step=0.5, value=0),
    tmax=wdg.SelectionSlider(options=[("%g" % i, i) for i in np.arange(1e-6, 5e-6, 1e-6)]),
    Bfield=wdg.FloatSlider(min=0.0, max=20.0, step=0.1, value=0.0, description="B (mT)"),
    k=wdg.SelectionSlider(options=[("%g" % i, i) for i in np.arange(0, 1e8, 1e5)]),
    J=wdg.FloatSlider(min=-20, max=20, step=0.1, value=0.0),
    D=wdg.FloatSlider(min=-20, max=0, step=0.1, value=0.0)
)
def update(kinetic_model="Haberkorn(S)", init="S", obs="S", hfc1=0, hfc2=0, tmax=1e-6, Bfield=0.0, k=0, J=0, D=0):
    sim = build_sim(hfc1, hfc2)
    haberkorn_kinetics(sim, init, obs, tmax, Bfield, k, J, D, kinetic_model)


interactive(children=(Dropdown(description='kinetic_model', options=('Haberkorn(S)', 'Haberkorn(T)', 'Haberkor…

---

### Relaxation

Spin relaxation is the process that drives the electron-nuclear spin system of the radical pair towards thermal equilibrium.
This destroys the spin coherence that is essential for magnetic field effects / magnetic resonance in radical pairs. 

We introduce the following relaxation mechanisms:
* J-modulation (Singlet-Triplet dephasing (STD)) 
* D-modulation (Dipolar modulation)
* Random fields relaxation (RFR)


### Modulation by fluctuations in distance

Electron-electron exchange (J) and dipolar (D) couplings both depend on the distance between the centres of the electron spin density of the two radicals (r). If the distance between the two radicals changes during the lifetime of the radical pair, a modulation of J or D can occur.

Typical modulation mechanisms:
* Electron hopping between two radical pairs (*e.g.*, cryptochrome).
* Flexible biradicals (*e.g.*, FAD).

<img src="figures/modulation.png" width="1000" height="300" align="center"/>

#### J-modulation

J-modulation can cause spin relaxation.
If fluctuations in the exchange interaction (J) occur on a faster timescale than the coherent spin dynamics, spin relaxation can transpire. 
The rate is given by $k_{STD}$:

$$
k_{STD} = 4\langle[(J(r(t)) - J_{av})^2]\rangle\tau_c
$$

$\tau_c$ = correlation time = timescale of fluctuations

Rates are typically between $10^6$ $s^{-1}$ to $10^9$ $s^{-1}$.

J-modulation relaxes the following density matrix elements:

$$
\langle S | \hat{\rho} | T_m \rangle
$$

It is therefore referred to as ‘singlet-triplet dephasing’:

$$
\hat{\hat{R}} = k_{STD} [\hat{Q}_S ⊗ \hat{Q}_T + \hat{Q}_T ⊗ \hat{Q}_S]
$$

#### D-modulation

D-modulation can also cause spin relaxation.
If fluctuations in the dipolar coupling (D) occur on a faster timescale than the coherent spin dynamics, spin relaxation can also emerge. The rate is given by kD:

$$
k_D = \langle[(D(r(t)) - D_{av})^2]\rangle\tau_c
$$

D-modulation does not relax the density matrix element,

$$
\langle T_+ | \hat{\rho} | T_- \rangle
$$

But all other density matrix elements and therefore, it leads to both ‘singlet-triplet dephasing’ and ‘triplet-triplet dephasing’:

$$
\hat{\hat{R}} = k_D [\frac{1}{9} \hat{Q}_S ⊗ \hat{Q}_{T+} + \frac{1}{9} \hat{Q}_{T+} ⊗ \hat{Q}_S +  \frac{1}{9} \hat{Q}_S ⊗ \hat{Q}_{T-} + \frac{1}{9} \hat{Q}_{T-} ⊗ \hat{Q}_S + \\ \frac{4}{9} \hat{Q}_S ⊗ \hat{Q}_{T0} + \frac{4}{9} \hat{Q}_{T0} ⊗ \hat{Q}_S + \hat{Q}_{T+} ⊗ \hat{Q}_{T0} + \\ \hat{Q}_{T0} ⊗ \hat{Q}_{T+} + \hat{Q}_{T-} ⊗ \hat{Q}_{T0} + \hat{Q}_{T0} ⊗ \hat{Q}_{T-}]
$$

### Random fields relaxation

‘Random fields’ relaxation (RFR) causes indefinite, uncorrelated molecular motions which cause the expectation values of all three spin angular momentum operators (Sx, Sy, Sz) of both electrons to relax independently with the same rate constant ($k_{RFR}$).
The spins relax isotropically and drive the singlet yield to zero (monotonically) as $k_{RFR}$ is increased and is given by:

$$
\hat{\hat{R}} = k_{RFR} [1.5 \times E ⊗ E - S_{Ax} ⊗ \tilde{S}_{Ax} - S_{Ay} ⊗ \tilde{S}_{Ay} - S_{Az} ⊗ \tilde{S}_{Az} \\ 
- S_{Bx} ⊗ \tilde{S}_{Bx} - S_{By} ⊗ \tilde{S}_{By} - S_{Bz} ⊗ \tilde{S}_{Bz}]
$$

Where $\tilde{}$ is the transpose.

In [ ]:
def relaxation_model(model, krelax):
    if model == "SingletTripletDephasing":
        return [SingletTripletDephasing(krelax)]
    elif model == "DipolarModulation":
        return [DipolarModulation(krelax)]
    elif model == "RandomFields":
        return [RandomFields(krelax)]
    else:
        return []


def relaxation(sim, init, obs, tmax, Bfield, k, k_relax, J, D, kinetics_model, relaxations_model):
    dB = 0.5
    B0 = np.arange(0.0, 20.0 + 1e-9, dB)
    b_idx = nearest_index(B0, Bfield)

    time = np.arange(0, tmax, 10e-9)

    kinetics = kinetic_model(kinetics_model, k)
    relaxations = relaxation_model(relaxations_model, k_relax)

    init_state = states[init]
    obs_state  = states[obs]

    H = sim.total_hamiltonian(B0=0, D=D, J=J)
    sim.apply_liouville_hamiltonian_modifiers(H, kinetics + relaxations)
    rhos = magnetic_field_loop(sim, init_state, time, H, B0, B_axis="z")
    product_probabilities = sim.product_probability(obs_state, rhos)

    sim.apply_hilbert_kinetics(time, product_probabilities, kinetics)
    k = kinetics[0].rate_constant if kinetics else 1.0
    product_yields, product_yield_sums = sim.product_yield(product_probabilities, time, k)

    x = time * 1e6 

    fig = plt.figure()
    plt.plot(x, product_probabilities[b_idx], linewidth=2, label=r"$P_i(t)$")
    plt.fill_between(x, product_yields[b_idx], alpha=0.2, label=r"$\Phi_i$")
    plt.xlabel("Time / $\mu s$", size=14)
    plt.ylabel("Probability", size=14)
    plt.ylim([0, 1])
    plt.legend(fontsize=14)
    fig.set_size_inches([5, 2.5])

    print(f"[{kinetics_model} | {relaxations_model}] PY(B={B0[b_idx]:.1f} mT) = {product_yield_sums[b_idx]:.6g}")

@wdg.interact(
    kinetic_model = ["Haberkorn(S)", "Haberkorn(T)", "HaberkornFree"],
    relaxation_model = ["SingletTripletDephasing", "DipolarModulation", "RandomFields"],
    init=["S", "T", "T+", "T0", "T-"],
    obs=["S", "T", "T+", "T0", "T-"],
    hfc1 = wdg.FloatSlider(min=0.0, max=2.0, step=0.5, value=0),
    hfc2 = wdg.FloatSlider(min=0.0, max=2.0, step=0.5, value=0),
    tmax=wdg.SelectionSlider(options=[("%g" % i, i) for i in np.arange(1e-6, 5e-6, 1e-6)]),
    Bfield=wdg.FloatSlider(min=0.0, max=20.0, step=0.1, value=0.0, description="B (mT)"),
    k=wdg.SelectionSlider(options=[("%g" % i, i) for i in np.arange(0, 1e8, 1e5)]),
    k_relax=wdg.SelectionSlider(options=[("%g" % i, i) for i in np.arange(0, 1e8, 1e5)]),
    J=wdg.FloatSlider(min=-20, max=20, step=0.1, value=0.0),
    D=wdg.FloatSlider(min=-20, max=0, step=0.1, value=0.0)
)
def update(kinetic_model="None", relaxation_model="None", init="S", obs="S", hfc1=0, hfc2=0, tmax=1e-6, Bfield=0.0, k=0, k_relax=0, J=0, D=0):
    sim = build_sim(hfc1, hfc2)
    relaxation(sim, init, obs, tmax, Bfield, k, k_relax, J, D, kinetic_model, relaxation_model)


interactive(children=(Dropdown(description='kinetic_model', options=('Haberkorn(S)', 'Haberkorn(T)', 'Haberkor…